In [ ]:
import os
from huggingface_hub import login
# load HF token from environment variable HF_TOKEN
login(token=os.getenv('HF_TOKEN'))

In [2]:
import os
import re
import json
import math
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
import sys, subprocess
def pip_install(package):
    try:
        __import__(package.split('==')[0].replace('-','_'))
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
for pkg in ['sentence-transformers','PyPDF2','scikit-learn','tqdm','huggingface_hub']:
    pip_install(pkg)
print('Dependencies ensured.')

Dependencies ensured.


In [4]:
from typing import List, Dict, Tuple, Optional, Iterable
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
try:
    from sentence_transformers import SentenceTransformer
    SENTENCE_TRANSFORMERS_AVAILABLE = True
except Exception:
    SENTENCE_TRANSFORMERS_AVAILABLE = False

try:
    from sentence_transformers import CrossEncoder
    CROSS_ENCODER_AVAILABLE = True
except Exception:
    CROSS_ENCODER_AVAILABLE = False
import PyPDF2
from huggingface_hub import InferenceClient

In [5]:
DATA_ROOT = Path('archive/supreme_court_judgments')
TARGET_CASE_FILENAME = 'Abdulla_Ahmed_vs_Animendra_Kissen_Mitter_on_14_March_1950_1.PDF'
YEARS = [y for y in os.listdir(DATA_ROOT) if y.isdigit()]
USE_HF_REASONING = True

REASONING_MODEL_ID = 'meta-llama/Llama-3.3-70B-Instruct'
HF_MAX_TOKENS = 400
HF_TEMPERATURE = 0.3
CHUNK_SIZE = 800
CHUNK_OVERLAP = 120
MAX_FILES_PER_YEAR = 20
USE_CROSS_ENCODER = False
CROSS_ENCODER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
PRIORITIZE_SUPREME = True
CITATION_BOOST = True

PRELIM_TOP_K = 10 
FINAL_TOP_K = 5
print(f'Years: {len(YEARS)} | HF reasoning: {USE_HF_REASONING} | Model: {REASONING_MODEL_ID.split("/")[-1][:30]}')

Years: 76 | HF reasoning: True | Model: Llama-3.3-70B-Instruct


In [6]:
def read_pdf_text(pdf_path: Path) -> str:
    text_parts = []
    try:
        with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                try:
                    page_text = page.extract_text() or ''
                except Exception:
                    page_text = ''
                text_parts.append(page_text)
    except Exception as e:
        print(f'Failed to read {pdf_path}: {e}')
    text = '\n'.join(text_parts)

    text = re.sub(r'[ \t\u00A0]+', ' ', text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text.strip()

In [7]:
SECTION_KEYS = ['case_title','date','court','bench','facts','issues','verdict_reasoning','citation']
def extract_sections(text: str) -> Dict[str, str]:

    data = {k: '' for k in SECTION_KEYS}
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    head = '\n'.join(lines[:50]) if lines else ''
    body = '\n'.join(lines)

    title_match = re.search(r'([A-Z][^\n]{5,100}?)\s+(v\.?|vs\.?|versus)\s+([^\n]{3,120})', head, flags=re.IGNORECASE)
    if title_match:
        data['case_title'] = (title_match.group(1)+' v. '+title_match.group(3)).strip()
    else:
    
        data['case_title'] = lines[0] if lines else ''

    date_match = re.search(r'(\b\d{1,2}\s+[A-Za-z]{3,9}\s+\d{4}\b|\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b)', head)
    if date_match:
        data['date'] = date_match.group(1)

    court_match = re.search(r'(SUPREME COURT OF INDIA|HIGH COURT[^\n]*|IN THE SUPREME COURT OF INDIA)', body, flags=re.IGNORECASE)
    if court_match:
        data['court'] = court_match.group(1).title()
    bench_match = re.search(r'CORAM\s*[:\-]\s*([^\n]+)', body, flags=re.IGNORECASE)
    if bench_match:
        data['bench'] = bench_match.group(1).strip()

    citation_match = re.search(r'\b(\d{4}\s*\(\d+\)\s*SCC\s*\d+|\d{4}\s*SCC\s*\(.*?\)\s*\d+|AIR\s*\d{4}\s*SC\s*\d+|\d{4}\s*SCR\s*\d+)\b', body)
    if citation_match:
        data['citation'] = citation_match.group(1)

    def extract_block(keyword_patterns: List[str]) -> str:

        pattern = re.compile('|'.join(keyword_patterns), re.IGNORECASE)
        idx = None
        for i, line in enumerate(lines):
            if pattern.search(line):
                idx = i
                break
        if idx is None:
            return ''

        block = []
        for j in range(idx+1, len(lines)):
            line = lines[j]
            if re.match(r'^[A-Z][A-Z\s\-]{2,60}$', line) and len(line.split()) <= 8:
                break
            block.append(line)
        return '\n'.join(block).strip()
    data['facts'] = extract_block(['FACTS','BACKGROUND','FACTUAL MATRIX']) or '\n'.join(lines[:200])
    data['issues'] = extract_block(['ISSUE','QUESTION(S) OF LAW','POINTS FOR DETERMINATION'])
    data['verdict_reasoning'] = extract_block(['HELD','JUDGMENT','REASON','CONCLUSION','ORDER','DECISION'])
    return data

In [8]:

class EmbeddingBackend:
    def __init__(self, model_name: str = 'sentence-transformers/all-MiniLM-L6-v2'):
        self.mode = 'sbert'
        self.model = None
        self.vectorizer = None
        if SENTENCE_TRANSFORMERS_AVAILABLE:
            try:
                self.model = SentenceTransformer(model_name)
                self.mode = 'sbert'
            except Exception as e:
                print(f'Failed to load SBERT model ({e}); falling back to TF-IDF.')
                self._init_tfidf()
        else:
            print('sentence-transformers not available; using TF-IDF fallback.')
            self._init_tfidf()
    def _init_tfidf(self):
        self.mode = 'tfidf'
        self.vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=60000)
    def fit_corpus(self, docs: List[str]):
        if self.mode == 'tfidf':
            self.vectorizer.fit(docs)
    def encode(self, docs: List[str]) -> np.ndarray:
        if self.mode == 'sbert' and self.model is not None:
            arr = self.model.encode(docs, normalize_embeddings=True, convert_to_numpy=True, batch_size=32)
            return arr
        X = self.vectorizer.transform(docs) if hasattr(self.vectorizer, 'vocabulary_') and self.vectorizer.vocabulary_ else self.vectorizer.fit_transform(docs)
        from sklearn.preprocessing import normalize
        return normalize(X).toarray()
def aggregate_chunk_vectors(vectors: np.ndarray, method: str = 'mean') -> np.ndarray:
    if method == 'max':
        return vectors.max(axis=0, keepdims=True)

    return vectors.mean(axis=0, keepdims=True)
print('Embedding backend (enhanced) ready.')

Embedding backend (enhanced) ready.


In [9]:

def gather_case_texts(limit_per_year: Optional[int]=None) -> List[Tuple[str,str,str]]:
    records = []
    for year in YEARS:
        year_path = DATA_ROOT / year
        pdfs = [f for f in os.listdir(year_path) if f.lower().endswith('.pdf')]
        if limit_per_year:
            pdfs = pdfs[:limit_per_year]
        for pdf in pdfs:
            path = year_path / pdf
            text = read_pdf_text(path)
            records.append((year, pdf, text))
    return records
def build_document_representation(sections: Dict[str,str]) -> str:
    ordered = [sections.get(k,'') for k in SECTION_KEYS]
    return '\n\n'.join([s for s in ordered if s])
print('Functions ready.')

Functions ready.


In [10]:

def process_target_case() -> Tuple[Dict[str,str], str, str]:
    target_path = DATA_ROOT / '1950' / TARGET_CASE_FILENAME
    raw = read_pdf_text(target_path)
    sections = extract_sections(raw)
    doc = build_document_representation(sections)
    return sections, doc, raw
def rank_similar(target_doc: str, corpus: List[Tuple[str,str,str]], top_k: int=5, target_raw: str = '', target_sections_arg: Optional[Dict[str,str]] = None) -> List[Dict[str,object]]:
    backend = EmbeddingBackend()
    target_vec = doc_embed(backend, target_doc)
    if target_sections_arg:
        target_sections = target_sections_arg
    else:
        target_sections = extract_sections(target_doc)
    target_cites = extract_citations(target_raw) if target_raw else extract_citations(target_doc)
    vectors = []
    meta = []
    texts_for_encode = []
    if backend.mode == 'tfidf':
        texts_for_encode = [build_document_representation(extract_sections(t)) for (_,_,t) in corpus]
        backend.fit_corpus([target_doc] + texts_for_encode)
        cand_matrix = backend.encode(texts_for_encode)
        target_vec = backend.encode([target_doc])
    for idx, (year, filename, text) in enumerate(corpus):
        if backend.mode == 'sbert':
            doc_text = build_document_representation(extract_sections(text))
            vec = doc_embed(backend, doc_text)
        else:
            vec = cand_matrix[idx:idx+1]
        vectors.append(vec)
        meta.append((year, filename, text))
    if not vectors:
        return []
    all_vecs = np.vstack(vectors)
    sims = cosine_similarity(target_vec, all_vecs)[0]
    prelim = list(zip(meta, sims))
    boosted = []
    for (year, filename, raw_text), base_score in prelim:
        secs = extract_sections(raw_text)
        new_score = boost_score(base_score, secs, target_cites, raw_text)
        boosted.append(((year, filename, raw_text), new_score, base_score))
    boosted.sort(key=lambda x: x[1], reverse=True)

    prelim_count = PRELIM_TOP_K if USE_HF_REASONING else top_k
    top = boosted[:prelim_count]
    if USE_CROSS_ENCODER and CROSS_ENCODER_AVAILABLE:
        try:
            cross_model = CrossEncoder(CROSS_ENCODER_MODEL)
            pairs = []
            for (year, filename, raw_text), boosted_score, base_score in top:
                cand_secs = extract_sections(raw_text)
                pairs.append((target_sections.get('issues','') or target_doc[:800], cand_secs.get('issues','') or raw_text[:800]))
            ce_scores = cross_model.predict(pairs)
            new = []
            for ((meta_triplet, boosted_score, base_score), ce) in zip(top, ce_scores):
                blend = 0.5*boosted_score + 0.5*float(ce)
                new.append((meta_triplet, blend, boosted_score, base_score))
            new.sort(key=lambda x: x[1], reverse=True)
            if new:
                ce_vals = [n[1] for n in new]
                mn, mx = min(ce_vals), max(ce_vals)
                rng = (mx-mn) or 1.0
                top_after_ce = []
                for (meta_triplet, blend, boosted_score, base_score) in new:
                    norm = (blend-mn)/rng
                    top_after_ce.append((meta_triplet, norm))
                top_candidates = top_after_ce[:prelim_count]
            else:
                top_candidates = [(m,b) for (m,b,_) in boosted[:prelim_count]]
        except Exception as e:
            print(f'Cross-encoder failed ({e}); falling back.')
            top_candidates = [(m,s) for (m,s,_) in boosted[:prelim_count]]
    else:
        top_candidates = [(m,s) for (m,s,_) in boosted[:prelim_count]]

    cand_data = []
    for (year, filename, raw_text), score in top_candidates:
        secs = extract_sections(raw_text)
        cand_data.append((secs, score, year, filename))
    if USE_HF_REASONING:
        explanations = batch_generate_explanations(target_sections, [(s,sc) for (s,sc,_,_) in cand_data])
    else:
        explanations = []
        for (secs, score, _, _) in cand_data:
            expl = generate_explanation(target_sections, secs, score)
            explanations.append(expl)

    results = []
    for idx, ((secs, score, year, filename), explanation) in enumerate(zip(cand_data, explanations)):
        if idx >= top_k:
            break
        results.append({
            'case_name': best_case_name(filename, secs),
            'year': year,
            'citation': secs.get('citation',''),
            'similarity': round(float(score),4),
            'reason': explanation
        })
    return results
print('Two-stage ranking pipeline ready.')

Two-stage ranking pipeline ready.


In [15]:
MAX_FILES_PER_YEAR = 20
target_sections, target_doc, target_raw = process_target_case()
print('Target case:', target_sections.get('case_title', 'Unknown'))
print(f'Retrieving top {PRELIM_TOP_K} candidates, then using LLM for final {FINAL_TOP_K}...')
corpus = gather_case_texts(limit_per_year=MAX_FILES_PER_YEAR)
corpus = [(y,f,t) for (y,f,t) in corpus if f != TARGET_CASE_FILENAME or y != '1950']
results = rank_similar(target_doc, corpus, top_k=FINAL_TOP_K, target_raw=target_raw, target_sections_arg=target_sections)
output = [{
    'case_name': r['case_name'],
    'year': r['year'],
    'citation': r['citation'],
    'similarity': r['similarity'],
    'reason': r['reason']
} for r in results]
print('\n=== RESULTS ===')
print(json.dumps(output, indent=2))

Target case: Abdulla Ahmed v. Animendra Kissen Mitter on 14 March, 1950
Retrieving top 10 candidates, then using LLM for final 5...

=== RESULTS ===
[
  {
    "case_name": "Nandkishore Lalbhai Mehta v. New Era Fabrics P.Ltd.& Ors on 8",
    "year": "2015",
    "citation": "2015 (9) SCC 755",
    "similarity": 0.8335,
    "reason": "The similarity between this case and the target case lies in the context of contract negotiation and the role of agents or brokers in concluding sales agreements. Both cases involve disputes over the terms of a sale and the obligations of the parties involved, with a focus on the authority of agents to act on behalf of their principals. The legal questions surrounding contract formation and the duties of agents are central to both cases."
  },
  {
    "case_name": "Thomson Press (India) Ltd v. Nanak Builders & Investrs.P.Ltd &",
    "year": "2013",
    "citation": "2013 (5) SCC 397",
    "similarity": 0.8318,
    "reason": "This case shares similarities with

In [11]:
def chunk_text(text: str, size: int = CHUNK_SIZE, overlap: int = CHUNK_OVERLAP) -> List[str]:
    cleaned = text
    chunks = []
    i = 0
    while i < len(cleaned):
        end = i + size
        segment = cleaned[i:end]
        if segment.strip():
            chunks.append(segment.strip())
        i = end - overlap
        if i < 0:
            break
    return chunks
print('Chunking utility ready.')

Chunking utility ready.


In [12]:
CITATION_REGEX = re.compile(r"\b(\d{4}\s*\(\d+\)\s*SCC\s*\d+|\d{4}\s*SCC\s*\(.*?\)\s*\d+|AIR\s*\d{4}\s*SC\s*\d+|\d{4}\s*SCR\s*\d+)\b")
def extract_citations(text: str) -> set:
    return set(CITATION_REGEX.findall(text))
def doc_embed(backend: EmbeddingBackend, text: str) -> np.ndarray:
    if backend.mode == 'sbert':
        chs = chunk_text(text, CHUNK_SIZE, CHUNK_OVERLAP) or [text]
        vecs = backend.encode(chs)
        return aggregate_chunk_vectors(vecs, 'mean')

    return backend.encode([text])
def boost_score(base: float, secs: Dict[str,str], target_cites: set, cand_text: str) -> float:
    score = base
    if PRIORITIZE_SUPREME and secs.get('court','').lower().find('supreme court') >= 0:
        score += 0.02  
    if CITATION_BOOST:
        cand_cites = extract_citations(cand_text)
        if cand_cites & target_cites:
            score += 0.02
    return float(max(0.0, min(1.0, score)))
print('Citation utilities and doc_embed ready.')

Citation utilities and doc_embed ready.


In [13]:
LEGAL_NOISE_PREFIX = re.compile(r'^(BENCH:|CITATION:|ACT:|CORAM:|APPEAL|CRL\.|S\.L\.P\.|JUDGMENT:?|ORDER:?)', re.IGNORECASE)
TOKEN_PATTERN = r'(?u)\b[a-zA-Z][a-zA-Z\-]{2,}\b'
def clean_section_text(s: str, max_chars: int = 1200) -> str:
    if not s:
        return ''
    lines = [ln.strip() for ln in s.splitlines() if ln.strip()]
    lines = [ln for ln in lines if not LEGAL_NOISE_PREFIX.match(ln)]
    text = ' '.join(lines)
    text = re.sub(r'\s+', ' ', text).strip()
    return text[:max_chars]
def get_key_terms(a: str, b: str, top_k: int = 12) -> Tuple[List[str], List[str], List[str]]:
    vec = TfidfVectorizer(stop_words='english', ngram_range=(1,2), token_pattern=TOKEN_PATTERN, min_df=1, max_features=4000)
    X = vec.fit_transform([a, b])
    terms = np.array(vec.get_feature_names_out())
    a_scores = X.toarray()[0]
    b_scores = X.toarray()[1]
    a_top_idx = np.argsort(a_scores)[::-1][:top_k]
    b_top_idx = np.argsort(b_scores)[::-1][:top_k]
    a_top = [t for t,s in zip(terms[a_top_idx], a_scores[a_top_idx]) if s>0][:top_k]
    b_top = [t for t,s in zip(terms[b_top_idx], b_scores[b_top_idx]) if s>0][:top_k]
    overlap = list(dict.fromkeys([t for t in a_top if t in set(b_top)]) )
    return a_top, b_top, overlap
def best_case_name(filename: str, sections: Dict[str,str]) -> str:
    if sections.get('case_title'):
        return sections['case_title'][:160]
    base = os.path.splitext(filename)[0].replace('_',' ')
    # Trim trailing ' on <date>' if present
    base = re.split(r'\s+on\s+\d', base, maxsplit=1)[0]
    return base[:160]
print('Cleaning and key term utilities ready.')

Cleaning and key term utilities ready.


In [ ]:
def batch_generate_explanations(target_sections: Dict[str,str], candidates: List[Tuple[Dict[str,str], float]]) -> List[str]:
    if not USE_HF_REASONING or not candidates:
        return ['No explanation available.' for _ in candidates]
    try:
        client = InferenceClient()
    except Exception as e:
        return [f'HF client init failed: {e}' for _ in candidates]
    t_title = target_sections.get('case_title','Unknown')
    t_date = target_sections.get('date','')
    t_court = target_sections.get('court','')
    t_issues = clean_section_text(target_sections.get('issues',''))[:800]
    t_reasoning = clean_section_text(target_sections.get('verdict_reasoning',''))[:800]
    cand_blocks = []
    for idx, (cand_sec, score) in enumerate(candidates, 1):
        c_title = cand_sec.get('case_title','Unknown')
        c_date = cand_sec.get('date','')
        c_court = cand_sec.get('court','')
        c_issues = clean_section_text(cand_sec.get('issues',''))[:700]
        c_reasoning = clean_section_text(cand_sec.get('verdict_reasoning',''))[:700]
        cand_blocks.append(f"""CANDIDATE {idx} (Similarity: {score:.3f}):
Title: {c_title}
Date: {c_date}
Court: {c_court}
Issues: {c_issues}
Reasoning: {c_reasoning}""")
    all_cands = '\n\n'.join(cand_blocks)
    system_msg = "You are an expert legal AI assistant analyzing Indian court judgments."
    user_msg = f"""Compare the TARGET case with each CANDIDATE case below and explain why they are similar in 2-3 precise sentences per candidate.

TARGET CASE:
Title: {t_title}
Date: {t_date}
Court: {t_court}
Issues: {t_issues}
Reasoning: {t_reasoning}

{all_cands}

INSTRUCTIONS:
- For each candidate, write exactly 2-3 sentences explaining the similarity.
- Focus on: shared factual background, overlapping legal questions, similar reasoning or doctrines applied.
- Be concrete and specific; avoid vague phrases like 'both deal with law' or 'similar structure'.
- Do not mention bench names, judge panels, or raw citation strings unless critical.
- Output format: Start each explanation with 'CANDIDATE X:' on its own line, then your 2-3 sentences.

Begin:"""
    try:
        resp = client.chat_completion(
            messages=[
                {'role':'system','content':system_msg},
                {'role':'user','content':user_msg}
            ],
            model=REASONING_MODEL_ID,
            max_tokens=HF_MAX_TOKENS*len(candidates),
            temperature=HF_TEMPERATURE
        )
        text = resp.choices[0].message.content.strip()
        explanations = []
        lines = text.split('\n')
        current = []
        for line in lines:
            if re.match(r'^CANDIDATE \d+:', line, re.IGNORECASE):
                if current:
                    explanations.append(' '.join(current).strip())
                current = []
            else:
                if line.strip():
                    current.append(line.strip())
        if current:
            explanations.append(' '.join(current).strip())
        while len(explanations) < len(candidates):
            explanations.append('Similarity explanation unavailable.')
        return explanations[:len(candidates)]
    except Exception as e:
        return [f'LLM reasoning failed: {str(e)[:200]}' for _ in candidates]
print('Batch LLM reasoning (chat API) ready.')

Batch LLM reasoning (chat API) ready.


In [ ]:
**Architecture:**
1. **Stage 1 (Retrieval):** Use SBERT embeddings + cosine similarity to retrieve top 10 candidate cases
2. **Stage 2 (Reasoning):** Send target + all 10 candidates to a powerful LLM (Llama-3.3-70B) to generate precise 2-3 sentence explanations for each, then return top 5

**Key Improvements:**
- Batch LLM reasoning with full context (target + all candidates in one prompt)
- Uses chat completion API for better instruction following
- Concrete explanations focusing on factual overlap, legal questions, and doctrinal reasoning
- No more generic previews or bench/citation noise

**Configuration (Cell 6):**
- `REASONING_MODEL_ID`: Switch between meta-llama, Qwen, Mixtral, or others
- `PRELIM_TOP_K`: Number of candidates to retrieve (default 10)
- `FINAL_TOP_K`: Number with LLM explanations to return (default 5)

**To scale up:** Increase `MAX_FILES_PER_YEAR` in runner cell or remove the limit to index the full archive.

# Interactive Legal Chatbot
Chat with an AI assistant about your similar cases. The chatbot has full context of the target case and all similar cases found, and can answer questions, compare cases, analyze legal principles, and provide insights.

In [ ]:
CHATBOT_MODEL_ID = 'meta-llama/Llama-3.3-70B-Instruct'
CHATBOT_MAX_TOKENS = 800
CHATBOT_TEMPERATURE = 0.4
MAX_CONVERSATION_HISTORY = 10

print(f'Chatbot configured with model: {CHATBOT_MODEL_ID}')

Chatbot configured with model: meta-llama/Llama-3.3-70B-Instruct


In [ ]:
def build_chatbot_context(target_sections: Dict[str,str], similar_cases: List[Dict[str,object]]) -> str:
    """Create a rich knowledge base from target and similar cases for the chatbot."""
    
    context_parts = ["=== TARGET CASE ==="]
    context_parts.append(f"Title: {target_sections.get('case_title', 'Unknown')}")
    context_parts.append(f"Date: {target_sections.get('date', 'Unknown')}")
    context_parts.append(f"Court: {target_sections.get('court', 'Unknown')}")
    context_parts.append(f"Citation: {target_sections.get('citation', 'N/A')}")
    context_parts.append(f"\nFacts:\n{clean_section_text(target_sections.get('facts', ''), 1000)}")
    context_parts.append(f"\nIssues:\n{clean_section_text(target_sections.get('issues', ''), 800)}")
    context_parts.append(f"\nReasoning/Verdict:\n{clean_section_text(target_sections.get('verdict_reasoning', ''), 1000)}")
    
    context_parts.append("\n\n=== SIMILAR CASES ===")
    for idx, case in enumerate(similar_cases, 1):
        context_parts.append(f"\n--- Case {idx} ---")
        context_parts.append(f"Name: {case['case_name']}")
        context_parts.append(f"Year: {case['year']}")
        context_parts.append(f"Citation: {case.get('citation', 'N/A')}")
        context_parts.append(f"Similarity Score: {case['similarity']}")
        context_parts.append(f"Why Similar: {case['reason']}")
    
    return '\n'.join(context_parts)

print('Context builder ready.')

Context builder ready.


In [ ]:
class LegalChatbot:
    """Interactive chatbot for discussing legal cases with context awareness."""
    
    def __init__(self, target_sections: Dict[str,str], similar_cases: List[Dict[str,object]], 
                 model_id: str = CHATBOT_MODEL_ID):
        self.target_sections = target_sections
        self.similar_cases = similar_cases
        self.model_id = model_id
        self.conversation_history = []
        self.context = build_chatbot_context(target_sections, similar_cases)
        
        try:
            self.client = InferenceClient()
            self.available = True
        except Exception as e:
            print(f"Warning: Could not initialize HF client: {e}")
            self.available = False
    
    def chat(self, user_message: str) -> str:
        """Send a message and get a response from the chatbot."""
        if not self.available:
            return "Chatbot is not available. Please check your Hugging Face token and internet connection."
        
        if not user_message.strip():
            return "Please enter a question or message."
        
        # Add user message to history
        self.conversation_history.append({'role': 'user', 'content': user_message})
        
        # Keep only recent history to avoid token limits
        if len(self.conversation_history) > MAX_CONVERSATION_HISTORY * 2:
            self.conversation_history = self.conversation_history[-(MAX_CONVERSATION_HISTORY * 2):]
        
        # Build system prompt with case context
        system_prompt = f"""You are an expert legal AI assistant helping lawyers and paralegals analyze Indian court judgments.

You have access to:
1. A TARGET case that the user is researching
2. Several SIMILAR cases that were found to be relevant

KNOWLEDGE BASE:
{self.context}

INSTRUCTIONS:
- Answer questions about the target case, similar cases, or comparisons between them
- Provide precise legal analysis based on the facts, issues, and reasoning in the cases
- When comparing cases, highlight specific similarities and differences
- Cite case names and citations when referencing specific cases
- If asked about legal principles, extract them from the reasoning sections
- Be concise but thorough; avoid speculation beyond what's in the case documents
- If you don't have enough information to answer, say so clearly
- Use professional legal language appropriate for lawyers/paralegals

Remember: You are analyzing real court judgments. Be accurate and professional."""

        # Build messages for API
        messages = [{'role': 'system', 'content': system_prompt}]
        messages.extend(self.conversation_history)
        
        try:
            response = self.client.chat_completion(
                messages=messages,
                model=self.model_id,
                max_tokens=CHATBOT_MAX_TOKENS,
                temperature=CHATBOT_TEMPERATURE
            )
            
            assistant_message = response.choices[0].message.content.strip()
            
            # Add assistant response to history
            self.conversation_history.append({'role': 'assistant', 'content': assistant_message})
            
            return assistant_message
            
        except Exception as e:
            error_msg = f"Error communicating with chatbot: {str(e)[:300]}"
            return error_msg
    
    def reset_conversation(self):
        """Clear conversation history."""
        self.conversation_history = []
        return "Conversation history cleared. Starting fresh!"
    
    def get_suggested_questions(self) -> List[str]:
        """Return sample questions users might ask."""
        return [
            "What are the key legal issues in the target case?",
            "Compare the reasoning in the target case with the most similar case.",
            "What legal principles are common across all these cases?",
            "Which case has the strongest precedential value and why?",
            "Summarize the facts of the target case.",
            "Are there any conflicting interpretations across these cases?",
            "What arguments could I make using these similar cases?",
            "What are the main differences between case 1 and case 2?",
            "Which cases involve constitutional interpretation?",
            "What remedies were granted in these cases?"
        ]

print('LegalChatbot class ready.')

LegalChatbot class ready.


In [41]:
# Initialize the chatbot with your results
chatbot = LegalChatbot(target_sections, results)
print("✓ Chatbot initialized successfully!")
print(f"✓ Target case: {target_sections.get('case_title', 'Unknown')}")
print(f"✓ Number of similar cases loaded: {len(results)}")
print("\n" + "="*60)
print("CHATBOT READY - You can now ask questions!")
print("="*60)

✓ Chatbot initialized successfully!
✓ Target case: Abdulla Ahmed v. Animendra Kissen Mitter on 14 March, 1950
✓ Number of similar cases loaded: 5

CHATBOT READY - You can now ask questions!


In [42]:
# Display suggested questions
print("💡 SUGGESTED QUESTIONS:")
print("-" * 60)
for i, question in enumerate(chatbot.get_suggested_questions(), 1):
    print(f"{i}. {question}")

💡 SUGGESTED QUESTIONS:
------------------------------------------------------------
1. What are the key legal issues in the target case?
2. Compare the reasoning in the target case with the most similar case.
3. What legal principles are common across all these cases?
4. Which case has the strongest precedential value and why?
5. Summarize the facts of the target case.
6. Are there any conflicting interpretations across these cases?
7. What arguments could I make using these similar cases?
8. What are the main differences between case 1 and case 2?
9. Which cases involve constitutional interpretation?
10. What remedies were granted in these cases?


In [43]:
# Example: Ask a question
question = "What are the key legal issues in the target case?"
print(f"\n🧑‍💼 USER: {question}\n")
response = chatbot.chat(question)
print(f"🤖 CHATBOT:\n{response}")


🧑‍💼 USER: What are the key legal issues in the target case?

🤖 CHATBOT:
The key legal issues in the target case, Abdulla Ahmed v. Animendra Kissen Mitter (1950 SCR 30), are:

1. **Scope of Authority of an Estate Broker**: The court examined the extent of the authority granted to the appellant (estate broker) by the respondent (principal) to negotiate the sale of the property. The commission letter authorized the appellant to "negotiate the sale" of the premises, and the court had to determine whether this authority empowered the broker to conclude a contract on behalf of the principal.

2. **Construction of Contractual Terms**: The court interpreted the terms of the commission letter and the subsequent correspondence between the parties to ascertain the intentions of the principal and the broker. The court considered whether the broker's actions, including finding a ready and willing purchaser and concluding a contract, were within the scope of his authority.

3. **Broker's Right to C

In [ ]:
# Interactive chat loop - Run this cell and keep asking questions!
# Type 'exit', 'quit', or 'bye' to stop

print("\n" + "="*60)
print("INTERACTIVE CHAT MODE")
print("="*60)
print("Type your questions below. Type 'exit', 'quit', or 'bye' to stop.")
print("Type 'reset' to clear conversation history.")
print("Type 'suggestions' to see suggested questions.")
print("-"*60 + "\n")

while True:
    try:
        user_input = input("🧑‍💼 YOU: ").strip()
        
        if not user_input:
            continue
            
        if user_input.lower() in ['exit', 'quit', 'bye', 'stop']:
            print("\n👋 Chatbot session ended. Thank you!")
            break
        
        if user_input.lower() == 'reset':
            print(f"\n🤖 CHATBOT: {chatbot.reset_conversation()}\n")
            continue
        
        if user_input.lower() in ['suggestions', 'suggest', 'help']:
            print("\n💡 SUGGESTED QUESTIONS:")
            for i, q in enumerate(chatbot.get_suggested_questions(), 1):
                print(f"   {i}. {q}")
            print()
            continue
        
        response = chatbot.chat(user_input)
        print(f"\n🤖 CHATBOT:\n{response}\n")
        print("-"*60 + "\n")
        
    except KeyboardInterrupt:
        print("\n\n👋 Chatbot session interrupted. Goodbye!")
        break
    except EOFError:
        print("\n\n👋 Chatbot session ended. Goodbye!")
        break
    except Exception as e:
        print(f"\n⚠️ Error: {e}\n")
        continue


INTERACTIVE CHAT MODE
Type your questions below. Type 'exit', 'quit', or 'bye' to stop.
Type 'reset' to clear conversation history.
Type 'suggestions' to see suggested questions.
------------------------------------------------------------



## Alternative: Single Question Mode
If you prefer to ask questions one at a time (without the interactive loop), use the cells below:

In [ ]:
# Ask a single question - modify and run this cell multiple times
my_question = "Compare the reasoning in the target case with the most similar case."

print(f"🧑‍💼 USER: {my_question}\n")
response = chatbot.chat(my_question)
print(f"🤖 CHATBOT:\n{response}")

In [ ]:
# View conversation history
print("📜 CONVERSATION HISTORY:")
print("="*60)
for i, msg in enumerate(chatbot.conversation_history):
    role = "🧑‍💼 USER" if msg['role'] == 'user' else "🤖 CHATBOT"
    content = msg['content'][:200] + "..." if len(msg['content']) > 200 else msg['content']
    print(f"\n{role}:\n{content}\n")
    print("-"*60)